# Open Addressing

Open addressing is another remedy to key collisions. Let $k_1$ and $k_2$ be two colliding keys and $k_1$ already in the hash table. Instead of putting the $k_2$ data into the chain of $k_1$, we find an empty bucket to hold the $k_2$ item. In open addressing, all elements occupy the hash table itself. That is, each table
entry contains either an element of the dynamic set or NIL. 
So at any point, the size of the table must be greater than or equal to the total number of keys (Note that we can increase table size by copying old data if needed). This approach is also known as closed hashing.

When searching for an element, we systematically examine table slots until either we find the desired
element or we have ascertained that the element is not in the table. No lists and no elements are stored outside the table, unlike in chaining. Thus, in open addressing,
the hash table can “fill up” so that no further insertions can be made; one
consequence is that the load factor can never exceed 1.

State-of-the-art hash tables are typically variations of open addressing with some improvements.

# Probing
 
To perform insertion using open addressing, we successively examine, or probe,
the hash table until we find an empty slot in which to put the key. Instead of being
fixed in the order $0, 1, \ldots, m-1$ (which requires $\Theta(n)$ search time), the sequence
of positions probed depends upon the key being inserted. To determine which slots
to probe, we extend the hash function to include the probe number (starting from 0)
as a second input. Thus, the hash function becomes
$$h: U \times \{0,1,\ldots, m-1\} \rightarrow \{0,1,\ldots, m-1\}.$$

With open addressing, we'd want that for every key $k$, the probe sequence
$h(k,0), h(k,1), \ldots, h(k,m-1)$ is a permutation of $0,1,\ldots, m-1$. In particular,
we assume uniform hashing: the probe sequence of each key
is equally likely to be any of the $m!$ permutations of $0,1,\ldots, m-1$. True uniform hashing is difficult to implement, and in practice suitable
approximations (such as double hashing, defined below) are used.
These approximations, while guaranteeing that $h(k,0), h(k,1), \ldots, h(k,m-1)$ is a permutation of 
$0,1,\ldots, m-1$ for each key $k$, are not uniform.

You recall that we allow keys to be strings of text as we do in dictionaries, but to create and use hash tables, we would need to convert text keys into numeric keys. Thus, for simplicity, it suffices to assume that from now on we will only deal with numeric keys.  

Let $h'$ be a hash function from a universe of integers for numeric keys to the hash space of $H = \{0,1,\ldots, m-1\}$, referred to as an auxiliary hash function. 

## Linear probing

Let $h(k,i) = (h'(k) + i)\text{ mod } m$ for $i \in H$.

Given a data point with key $k$, to store data in the hash table, we first compute $j = h(k,0)$. If bucket $j$ is
already occupied, we compute $h(k,1)$, if the bucket $h(k,1)$ is also occupied, we compute $h(k,2)$, and repeat until an empty bucket is found. It's evident that if there is an empty bucket, then it can be found by linear probing. Likewise, to search for the data with key $k$, we repeat the same procedure, only this time we check for the key of the data point in the bucket, not to check if the bucket is occupied as in the storing phase. 

When the hash table is full, we will make a new larger hash table that includes the old table. How to do so effectively and efficiently is the topic of dynamic tables to be discussed later.

<!--In linear probing, the probe sequences of two colloding keys are the same, which is fine but both insertion and search operations might take a long time in the worst case. Imagine that in a half-full table, to probe sequence may happen to land on the occupied buckets in the first half of the probes, which takes linear time.
-->

## Quadratic probing

Let $h(k,i) = (h'(k) + ai + b i^2) \text{ mod } m$ for $i \in H$.

The constants $a$ and $b$ must be chosen carefully for the probe sequence to cover all the buckets. When the size
of the hash table is a power of 2, setting $a = b = 1/2$ guarantees that the probe sequence will cover all the
buckets before it starts repeating them. 

Quadratic probing offers a much better efficiency than linear probing. But it still has the same issue of having the same probing sequence for colliding keys. Namely, if $h(k_1,0) = h(k_2,0) = h'(k)$, then $h(k_1, i) = h(k_2,i)$ for any $i$.

## Double hashing probing

Doublie hashing probing is the best probing algorithm. It takes two auxilliary hash functions $h_1$ and $h_2$ with $h_1$ being the primary and $h_2$ the secondary. Let
$$h(k,i) = (h_1(k) + i h_2(k))\text{ mod } m.$$

We can see that even if $h(k_1, 0) = h(k_2,0)$, it is likely that $h(k_1,1) \not= h(k_2,1)$ as long as $h_2$ is a unifomrly distributed hash function.

Double hashing offers one of the best methods available for open addressing because
the permutations produced have many of the characteristics of randomly
chosen permutations.


# Insertion

<code>
HASH-INSERT(T,k)
1 i = 0
2 repeat
3      j = h(k,i)
4      if T[j]  == NIL or T[j] == "DELETED"
5         T[j] = k
6         return j
7      else i = i + 1
8 until i == m
9 error “hash table overflow”
</code>

# Search

<code>
HASH-SEARCH(T,k)
1 i = 0
2 repeat
3      j = h(k,i)
4      if T[j] == k
5         return j
6      i = i + 1
7 until T[j] == NIL or i == m
8 return NIL    
</code>

# Deletion

Deletion from an open-address hash table is not as straightforward. When we delete a key
from slot $i$ , we cannot simply mark that slot as empty by storing NIL in it. If
we did, we might be unable to retrieve any key $k$ during whose insertion we had
probed slot $i$ and found it occupied. We can solve this problem by marking the
slot, storing in it the special value DELETED instead of NIL. We would then modify
the procedure HASH-INSERT to treat such a slot as if it were empty so that we can
insert a new key there. We do not need to modify HASH-SEARCH, since it will pass
over DELETED values while searching. When we use the special value DELETED,
however, search times no longer depend on the load factor $\alpha$, and for this reason
chaining is more commonly selected as a collision resolution technique when keys
must be deleted.

<code>
HASH-DELETE(T,k)
1 i = 0
2 repeat
3      j = h(k,i)
4      if T[j] == k    
5         T[j] = "DELETED"
6         return
7      i = i + 1
8 until T[j] == NIL or i == m
9 print("item not found")   
</code>

# Analysis

## Inserting an element

Regardless what probing mechanism is used, assuming that the underlying hash function is uniform, the expected number of probes for inserting an element into the hash table is at most $m/(m -n) = 1/(1-\alpha)$ with $\alpha = n/m < 1$. This statement can be proven below:

Let $X$ be a random variable representing the number of probes made in an unsuccessful search. Let $A_i$ denote the event that an $i$-th probe occurs but it fails (i.e., the $i$-th probe hits an occupied slot). Then 
$\{X \geq i\} = A_1 \cap A_2 \cap \cdots A_{i-1}.$ Hence,
\begin{eqnarray*}
\text{Pr}\{X\geq i\} &=& \text{Pr}(A_1 \cap A_2 \cap \cdots A_{i-1}) \\
&=&\text{Pr}(A_1)\cdot \text{Pr}(A_2|A_1) \cdot \text{Pr}(A_3|A_1\cap A_2) \cdots \text{Pr}(A_{i-1}|A_1\cap \cdots A_{i-2}).
\end{eqnarray*}
We have $\text{Pr}(A_1) = n/m$, $\text{Pr}(A_2) = (n-1)/(m-1), \ldots, \text{Pr}(A_j) = (n-j+1)/(m-j+1)$. 
By assumption, $n < m$, and so for any $k>0$ we have $nk < mk$. For $0<k \leq n$, we have
$(n-k)/(m-k) < n/m$. Thus,
\begin{align*}
\text{Pr}\{X\geq i\} &= \frac{n}{m} \frac{n-1}{m-1} \cdots \frac{n-i+2}{m-i+2} \\
&\leq \left(\frac{n}{m}\right)^{i-1} \\
&= \alpha^{i-1}.
\end{align*}
Since there are only $m$ slots available, $\text{Pr}\{X\geq i\} = 0$ for $i > m$. Hence,
\begin{align*}
E[X] &= \sum_{i=1}^\infty \text{Pr}\{X\geq i\}\\
&= \sum_{i=1}^m \text{Pr}\{X\geq i\} \\
&\leq \sum_{i=1}^m \alpha^{i-1} \\
&= \frac{1-\alpha^i}{1-\alpha} \\
&< \frac{1}{1-\alpha}.
\end{align*}
This completes the proof.

Thus, inserting an element into an open-address hash table with load factor $\alpha = n/m < 1$ requres 
$i$ probes for some $i$ which is successful at the $(i+1)$-th probe. The expected number of probes
is at most
$1/(1-\alpha)$, assuming uniform hashing. 

Note that 
$$\frac{1}{1-\alpha} = 1 + \alpha + \alpha^2 + \cdots,$$
which provides an intuitive interpretation: $\alpha^i$ is the probability that the first $i$ probes fail to find an empty slot. In other words, with probability $\alpha$, the first probe fails, so a second probe is needed. With probability $\alpha^2$, the second probe also fails, so a third probe is needed, and so on.

## Searching

What is the expected number of probes in a successful search? It turns out that it is at most 
$$\frac{1}{\alpha}\ln \frac{1}{1-\alpha},$$
assuming uniform hashing and each key in the table being equally likely to be searched for. This statement can be 
proven as follows:

We have already proven that if $k$ was the $(i+1)$-th key inserted into the hash table, meaning that there are $i$ slots occupied among $m$ slots with a load factor $i/m$, the expected number of
probes made in a search for $k$ is the same as that to insert $k$, which is at most
$1/(1-i/m) = m/(m-i)$. 
Thus, the expected number of probes for a successful search is
\begin{align*}
\frac{1}{n}\sum_{i=0}^{n-1}\frac{m}{m-i}  
&= \frac{m}{n}\sum_{k=m-n+i}^m \frac{1}{k} \\
&\leq \frac{1}{\alpha}\int_{m-n}^m \frac{dx}{x} \\
&= \frac{1}{\alpha}\ln\frac{m}{m-n} \\
&= \frac{1}{\alpha}\ln\frac{1}{1-\alpha}.
\end{align*}
This completes the proof.


# Open addressing using perturbation probing

Python uses perturbation probing of the hash value as follows:
$$h(k) = 5h'(k) \wedge mask + 1 + h'(k) >> 5,$$
where $mask$ is a binary integer to mask out bits in $5h'(k)$, and $h'(k) >> 5$ means to shift the binary form of $h'(k)$ to the right by 5 places. Unlike previous probing methods that takes a second parameter $i$ as input, it uses right shift by 5 places in the following way:

<code>
h(k):
    first_probe = h'(k) & mask
    yield first_probe
    perturb = h'(k)
    while True:
        yield (first_probe * 5 + 1 + purturb) & mask
        perturb >>= 5
</code>

In [1]:
# Implementation of Dictionary in Python using perturbation probing

from typing import Tuple, Iterable, Any, List, Optional, Union
import array
import copy

DUMMY = -2
FREE = -1

class DictKey:
    def __init__(self, key, hashvalue):
        self.key = key
        self.hashvalue = hashvalue

    def __repr__(self):
        return f"<DictKey {self.key}>"

class Dictionary:
    __version = 0

    def __init__(self, *args, **kwargs):
        self._increase_version()
        self.Indices = self._make_index_array(8)  # init with an 8 elements table
        self.keys: List[DictKey] = []
        self.values: List[Any] = []
        self.used = 0  # number of items in the dictionary
        self.filled = 0  # number of non-empty slots including dummy slots
        self.update(*args, **kwargs)

    def _increase_version(self):
        """Increase version of current instance and of class attribute"""
        self.__version = Dictionary.__version
        Dictionary.__version += 1

    @staticmethod
    def _make_index_array(n: int) -> Union[list, array.array]:
        "New sequence of indices using the smallest possible datatype"
        if n <= 2 ** 7:
            return array.array("b", [FREE]) * n  # signed char
        if n <= 2 ** 15:
            return array.array("h", [FREE]) * n  # signed short
        if n <= 2 ** 31:
            return array.array("l", [FREE]) * n  # signed long
        return [FREE] * n

    @staticmethod
    def _generate_probes(hash_: int, mask: int) -> Iterable[int]:
        index = hash_ & mask
        yield index
        """ 
        The yield statement suspends a function’s execution and sends a value back to 
        the caller, but retains enough state to enable the function to resume where it 
        left off. When the function resumes, it continues execution immediately after 
        the last yield run. This allows its code to produce a series of values over time, 
        rather than computing them at once and sending them back like a list. 
        """ 
        perturb = hash_
        while True:
            new_hash = index * 5 + 1 + perturb
            yield new_hash & mask
            perturb >>= 5 # right shift by 5 places and assigns the new value to perturb

    def _lookup(self, key: Any, hashvalue: int) -> Tuple[int, Any]: # look for a free slot
        mask = len(self.Indices) - 1
        freeslot = None
        for index in self._generate_probes(hashvalue, mask): # generate an index
            entry_index = self.Indices[index]
            if entry_index == FREE:
                return (index, FREE) if freeslot is None else (freeslot, DUMMY)
            elif entry_index == DUMMY:
                if freeslot is None:
                    freeslot = index # update freeslot
            else:
                dict_key = self.keys[entry_index]
                if dict_key.key is key or (
                    dict_key.hashvalue == hashvalue and dict_key.key == key
                ):
                    return (index, entry_index)

    def update(self, other=(), /, **kwargs):
        """
        1. A slash in the argument list of a function denotes that the parameters prior to it 
        are positional-only. Positional-only parameters are the ones without an externally-usable 
        name. Upon calling a function that accepts positional-only parameters, arguments are 
        mapped to parameters based solely on their position.
        2. **kwargs in function definitions is used to pass a keyworded, variable-length argument list
        """
 
        self._sharing_keys = False
        if isinstance(other, Dictionary):
            if self.used > 0:
                for key in other:
                    self[key] = other[key]
            else:
                self._sharing_keys = True
                self.Indices = copy.copy(other.Indices) # shallow copy
                self.keys = other.keys
                self.values = [None] * len(other.values)
                self.used = other.used
                self.filled = other.filled
        elif hasattr(other, "keys"):
            for key in other.keys():
                self[key] = other[key]
        else:
            for key, value in other:
                self[key] = value
        for key, value in kwargs.items():
            self[key] = value

    def _check_keys_sharing(self):
        """if trying to change dictionary with shared keys,
        migrate to non shared dictionary"""
        if self._sharing_keys:
            self.keys = copy.copy(self.keys)

    def __getitem__(self, key: Any) -> Any: #search
        hashvalue = hash(key)
        _, entry_index = self._lookup(key, hashvalue)
        if entry_index < 0:  # FREE or DUMMY
            raise KeyError(key)
        return self.values[entry_index]

    def __setitem__(self, key: Any, value: Any):
        hashvalue = hash(key)
        indices_index, entry_index = self._lookup(key, hashvalue)
        if entry_index < 0:  # FREE or DUMMY
            self._check_keys_sharing()
            self._increase_version()
            self.Indices[indices_index] = self.used
            dict_key = DictKey(key=key, hashvalue=hashvalue)
            self.keys.append(dict_key)
            self.values.append(value)
            self.used += 1
            if entry_index == FREE:
                self.filled += 1  # DUMMY? `filled` would have already contained it
                if self.filled / len(self.Indices) > 2 / 3:
                    self._resize(4 * len(self))
        else:
            if value != self.values[entry_index]:  # only if its a different value
                self._increase_version()
                self.values[entry_index] = value

    def __delitem__(self, key: Any):
        self._check_keys_sharing()
        hashvalue = hash(key)
        indices_index, entry_index = self._lookup(key, hashvalue)
        if entry_index < 0:
            raise KeyError(key)

        self._increase_version()
        self.used -= 1
        self.Indices[indices_index] = DUMMY
        # del self.entries[entry_index]
        # swap with the last item to avoid holes
        if entry_index != self.used:
            last_key_dict = self.keys[-1]
            last_value = self.values[-1]
            last_entry_indices_index, _ = self._lookup(
                last_key_dict.key, last_key_dict.hashvalue
            )
            self.Indices[last_entry_indices_index] = entry_index
            self.keys[entry_index] = last_key_dict
            self.values[entry_index] = last_value
        self.keys.pop()
        self.values.pop()

    def _resize(self, n: int):
        n = 2 ** n.bit_length()
        self.Indices = self._make_index_array(n)
        for entry_index, dict_key in enumerate(self.keys):
            for i in self._generate_probes(dict_key.hashvalue, n - 1):
                if self.Indices[i] == FREE:
                    break
            self.Indices[i] = entry_index
        self.filled = self.used

    def __contains__(self, key: Any) -> bool:
        _, entry_index = self._lookup(key, hash(key))
        return entry_index >= 0

    def __len__(self) -> int:
        return self.used

    def __iter__(self):
        return iter([dict_key.key for dict_key in self.keys])

    def __repr__(self) -> str:
        return f"<Dictionary keys={self.keys}, values={self.values}, version={self.__version}>"

    def show(self):
        """Used for nicely dispalying the dictionary"""
        print("=" * 50)
        print(f"Dictionary version {self.__version}")
        print("-" * 50)
        print(f"Indices: {list(self.Indices)}")
        for i, row in enumerate(zip(self.keys, self.values)):
            print(i, row)
        print("=" * 50)



In [2]:
# Diver code
if __name__ == "__main__":
    d = Dictionary(
        [("key1", "value1"), ("key2", "value2"), (9, "different type of key")]
    )
    d.show()
    d["key1"] = "changed_value"
    d.show()
    del d["key2"]
    d.show()
    new_d = Dictionary(d)
    new_d["key1"] = "new_d_value"
    new_d[9] = "foobar"
    new_d.show()
    new_d["different_key"] = "value"
    new_d.show()
    new_d["foo"] = "bazz"
    new_d.show()
    d.show()
    new_d.update(d)
    new_d.show()
    new_d["more"] = "values"
    new_d["should"] = "resize"
    new_d.show()

Dictionary version 3
--------------------------------------------------
Indices: [0, 2, -1, 1, -1, -1, -1, -1]
0 (<DictKey key1>, 'value1')
1 (<DictKey key2>, 'value2')
2 (<DictKey 9>, 'different type of key')
Dictionary version 4
--------------------------------------------------
Indices: [0, 2, -1, 1, -1, -1, -1, -1]
0 (<DictKey key1>, 'changed_value')
1 (<DictKey key2>, 'value2')
2 (<DictKey 9>, 'different type of key')
Dictionary version 5
--------------------------------------------------
Indices: [0, 1, -1, -2, -1, -1, -1, -1]
0 (<DictKey key1>, 'changed_value')
1 (<DictKey 9>, 'different type of key')
Dictionary version 8
--------------------------------------------------
Indices: [0, 1, -1, -2, -1, -1, -1, -1]
0 (<DictKey key1>, 'new_d_value')
1 (<DictKey 9>, 'foobar')
Dictionary version 9
--------------------------------------------------
Indices: [0, 1, -1, -2, -1, -1, -1, 2]
0 (<DictKey key1>, 'new_d_value')
1 (<DictKey 9>, 'foobar')
2 (<DictKey different_key>, 'value')
Dict